# single_neuron_processing_one_hot
Assesses neuron by neuron data for each recording and then saves into summary files
- uses a one-hot encoding regression to identify selective neurons

In [ ]:
# imports 
import numpy as np 
import pandas as pd 
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
from pathlib import Path
import matplotlib.colors as mcolors
import warnings
import pingouin as pg
from sklearn.linear_model import LinearRegression
import statsmodels.formula.api as smf
import glob
import os
warnings.filterwarnings("ignore", message="Mean of empty slice", category=RuntimeWarning)

In [ ]:
# functions
def get_labelled_posteriors(indata, labels):

    '''
    INPUTS:
    indata = posterior probabilites from a classifier with the shape
            n_trials x n_timesteps x n_classes
        
    labels = 1d array with len(n_trials) - these labels ought
            to correspond to class numbers (layers in indata)

    OUTPUT:
        labelled_posteriors = posterior probabilities associated with the
        classes in the labels input for each timestep and trial
    '''

    n_trials, n_times, n_classes = indata.shape
    class_lbls = np.unique(labels)
    class_lbls = class_lbls[~np.isnan(class_lbls)]

    # initialize output
    labelled_posteriors = np.zeros(shape = (n_trials, n_times))

    for ix, lbl in enumerate(class_lbls):
        
        # find trials where this label was chosen
        labelled_posteriors[labels == lbl,:] = indata[labels == lbl,:,int(ix)]
        
    return labelled_posteriors


def pull_balanced_train_set(trials2balance, params2balance):
    '''
    INPUTS:
    trials2balance   - ***logical array*** of the trials you want to balance
    params2balance   - ***list*** where each element is a vector of categorical
                        parameters to balance (e.g. choice value and side)
                        each element of params2balance must have the same
                        number of elements as trials2balance
    OUTPUTS:
    train_ix         - trial indices of a fully balanced training set
    leftover_ix      - trial indices of trials not included in train_ix
    '''

    # Find the indices where trials are selected to balance
    balance_indices = np.where(trials2balance)[0]

    # Create an array of parameters to balance
    params_array = np.array(params2balance).T

    # Find unique combinations and their counts
    p_combos, p_counts = np.unique(params_array[balance_indices], axis=0, return_counts=True)

    # Determine the minimum count for a balanced set
    n_to_keep = np.min(p_counts)

    # Initialize arrays to mark selected and leftover trials
    train_ix = np.zeros(len(trials2balance), dtype=bool)
    leftover_ix = np.zeros(len(trials2balance), dtype=bool)

    # Select a balanced number of trials for each unique parameter combination
    for combo in p_combos:
        # Find indices of trials corresponding to the current combination
        combo_indices = np.where((params_array == combo).all(axis=1) & trials2balance)[0]

        # Shuffle the indices
        np.random.shuffle(combo_indices)

        # Select n_to_keep trials and mark them as part of the training set
        train_ix[combo_indices[:n_to_keep]] = True

        # Mark the remaining trials as leftovers
        leftover_ix[combo_indices[n_to_keep:]] = True

    return train_ix, leftover_ix


def random_prop_of_array(inarray, proportion):
    '''
    INPUTS
    inarray = logical/boolean array of indices to potentially use later
    proportion = how much of inarray should randomly be selected

    OUTPUT
    out_array = logical/boolean that's set as 'true' for a proportion of the 
                initial 'true' values in inarray
    '''

    out_array = np.zeros(shape = (len(inarray), ))

    # find where inarray is true and shuffle those indices
    shuffled_ixs = np.random.permutation(np.asarray(np.where(inarray)).flatten())

    # keep only a proportion of that array
    kept_ix = shuffled_ixs[0: round(len(shuffled_ixs)*proportion)]

    # fill in the kept indices
    out_array[kept_ix] = 1

    # make this a logical/boolean
    out_array = out_array > 0

    return out_array


def pull_from_h5(file_path, data_to_extract):
    try:
        with h5py.File(file_path, 'r') as file:
            # Check if the data_to_extract exists in the HDF5 file
            if data_to_extract in file:
                data = file[data_to_extract][...]  # Extract the data
                return data
            else:
                print(f"'{data_to_extract}' not found in the file.")
                return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
def list_hdf5_data(file_path):
    try:
        with h5py.File(file_path, 'r') as file:
            print(f"Datasets in '{file_path}':")
            for dataset in file:
                print(dataset)
    except Exception as e:
        print(f"An error occurred: {e}")


def get_ch_and_unch_vals(bhv):
    """
    Extracts chosen (ch_val) and unchosen (unch_val) values associated with each trial.

    Parameters:
    - bhv (DataFrame): DataFrame behavioral data.

    Returns:
    - ch_val (ndarray): Array of chosen values for each trial.
    - unch_val (ndarray): Array of unchosen values for each trial. 
                          - places 0s for unchosen values on forced choice trials
    """
    ch_val = np.zeros(shape=(len(bhv, )))
    unch_val = np.zeros(shape=(len(bhv, )))

    bhv['r_val'] = bhv['r_val'].fillna(0)
    bhv['l_val'] = bhv['l_val'].fillna(0)

    ch_left = bhv['side'] == -1
    ch_right = bhv['side'] == 1

    ch_val[ch_left] = bhv['l_val'].loc[ch_left].astype(int)
    ch_val[ch_right] = bhv['r_val'].loc[ch_right].astype(int)

    unch_val[ch_left] = bhv['r_val'].loc[ch_left].astype(int)
    unch_val[ch_right] = bhv['l_val'].loc[ch_right].astype(int)

    return ch_val, unch_val


def get_ch_and_unch_pps(in_pp, bhv, ch_val, unch_val):
    """Gets the posteriors associated with the chosen and unchosen classes

    Args:
        in_pp (ndarray): array of posteriors (n_trials x n_times x n_classes)
        bhv (dataframe): details of each trial
        ch_val (ndarray): vector indicating the class that is ultimately chosen
        unch_val (ndarray): vector indicating the class that was ultimately not chosen

    Returns:
        ch_pp (ndarray): vector of the postior at each point in time for each trial's chosen option
        unch_pp (ndarray): vector of the postior at each point in time for each trial's unchosen option
    """

    # select the chosen and unchosen values 
    n_trials, n_times, n_classes = np.shape(in_pp)
    ch_pp = np.zeros(shape=(n_trials, n_times))
    unch_pp = np.zeros(shape=(n_trials, n_times))

    # loop over each trial
    for t in range(n_trials):
        
        # get the chosen and unchosen PPs
        ch_pp[t, :] = in_pp[t, :, int(ch_val[t]-1)]
        unch_pp[t, :] = in_pp[t, :, int(unch_val[t]-1)]
        
    # set the forced choice unchosen pps to nans, since there was only 1 option
    unch_pp[bhv['forced'] == 1, :] = np.nan
    
    return ch_pp, unch_pp


def get_alt_ch_and_unch_pps(in_pp, bhv, s_ch_val, s_unch_val):
    """Gets the posteriors associated with the chosen and unchosen classes

    Args:
        in_pp (ndarray): array of posteriors (n_trials x n_times x n_classes)
        bhv (dataframe): details of each trial
        s_ch_val (ndarray): vector indicating the class that is ultimately chosen
        s_unch_val (ndarray): vector indicating the class that was ultimately not chosen

    Returns:
        alt_ch_pp (ndarray): vector of the postior at each point in time for the alternative value in the other state
        alt_unch_pp (ndarray): vector of the postior at each point in time for the alternative value in the other state
    """

    # select the chosen and unchosen values 
    n_trials, n_times, n_classes = np.shape(in_pp)
    alt_ch_pp = np.zeros(shape=(n_trials, n_times))
    alt_unch_pp = np.zeros(shape=(n_trials, n_times))

    alt_ch_val = np.zeros_like(s_ch_val)
    alt_unch_val = np.zeros_like(s_unch_val)
    
    alt_ch_val[bhv['state'] == 1] = 8 - s_ch_val[bhv['state'] == 1] + 1
    alt_ch_val[bhv['state'] == 2] = 8 - s_ch_val[bhv['state'] == 2] + 1

    alt_unch_val[bhv['state'] == 1] = 8 - s_unch_val[bhv['state'] == 1] + 1
    alt_unch_val[bhv['state'] == 2] = 8 - s_unch_val[bhv['state'] == 2] + 1

    for t in range(n_trials):
        
        alt_ch_pp[t, :] = in_pp[t, :, int(alt_ch_val[t]-1)]
        alt_unch_pp[t, :] = in_pp[t, :, int(alt_unch_val[t]-1)]

    # set the alternative values to nans for state 3, since there were no alternatives
    alt_ch_pp[bhv['state'] == 3] = np.nan
    alt_unch_pp[bhv['state'] == 3] = np.nan

    return alt_ch_pp, alt_unch_pp

def find_candidate_states(indata, n_classes, temporal_thresh, mag_thresh):
    """Finds periods where decoded posteriors are twice their noise level.

    Args:
        indata (ndarray): 2d array of posterior probabilities associated with some decoder output.
        n_classes (int): How many classes were used in the decoder?
        temporal_thresh (int): Number of contiguous samples that must be above a threshold to be a real state (typically 2).
        mag_thresh (flat): how many times the noise level must a state be? (e.g. 2 = twice the noise level)

    Returns:
        state_details (ndarray): 2d array where each row details when a state occurred [trial_num, time_in_trial, state_length].
        state_array (ndarray): 2d array the same size as indata. It contains 1 in all locations where there were states and 0s everywhere else.
    """
    state_details = np.array([])
    state_array = np.zeros_like(indata)
    
    state_magnitude_thresh = (1 / n_classes) * mag_thresh

    for t in range(indata.shape[0]):
        state_len, state_pos, state_type = find_1dsequences(indata[t, :] > state_magnitude_thresh)
        state_len = state_len[state_type == True]
        state_pos = state_pos[state_type == True]

        for i in range(len(state_len)):
            state_details = np.concatenate((state_details, np.array([t, state_pos[i], state_len[i]])))

    state_details = state_details.reshape(-1, 3)
    state_details = state_details[state_details[:, 2] > temporal_thresh, :]

    # Update state_array using state_details information
    for j in range(len(state_details)):
        state_trial, state_start, state_len = state_details[j].astype(int)
        state_array[state_trial, state_start:(state_start + state_len)] = 1

    return state_details, state_array

def moving_average(x, w, axis=0):
    '''
    Moving average function that operates along specified dimensions of a NumPy array.

    Parameters:
    - x (numpy.ndarray): Input array.
    - w (int): Size of the window to convolve the array with (i.e., smoothness factor).
    - axis (int): Axis along which to perform the moving average (default is 0).

    Returns:
    - numpy.ndarray: Smoothed array along the specified axis with the same size as the input array.
    '''
    x = np.asarray(x)  # Ensure input is a NumPy array
    if np.isnan(x).any():
        x = np.nan_to_num(x)  # Replace NaN values with zeros

    if axis < 0:
        axis += x.ndim  # Adjust negative axis value

    kernel = np.ones(w) / w  # Create kernel for moving average

    # Pad the array before applying convolution
    pad_width = [(0, 0)] * x.ndim  # Initialize padding for each axis
    pad_width[axis] = (w - 1, 0)  # Pad along the specified axis (left side)
    x_padded = np.pad(x, pad_width, mode='constant', constant_values=0)

    # Apply 1D convolution along the specified axis on the padded array
    return np.apply_along_axis(lambda m: np.convolve(m, kernel, mode='valid'), axis, x_padded)

def find_1dsequences(inarray):
        ''' 
        run length encoding. Partial credit to R rle function. 
        Multi datatype arrays catered for including non Numpy
        returns: tuple (runlengths, startpositions, values) 
        '''
        ia = np.asarray(inarray)                # force numpy
        n = len(ia)
        if n == 0: 
            return (None, None, None)
        else:
            y = ia[1:] != ia[:-1]                 # pairwise unequal (string safe)
            i = np.append(np.where(y), n - 1)     # must include last element 
            lens = np.diff(np.append(-1, i))      # run lengths
            pos = np.cumsum(np.append(0, lens))[:-1] # positions
            return(lens, pos, ia[i])
        
        
def calculate_mean_and_interval(data, type='sem', num_samples=1000, alpha=0.05):
    """
    Calculate mean and either SEM or bootstrapped CI for each column of the input array, disregarding NaN values.

    Parameters:
    - data: 2D numpy array
    - type: str, either 'sem' or 'bootstrap_ci'
    - num_samples: int, number of bootstrap samples (applicable only for type='bootstrap_ci')
    - alpha: float, significance level for the confidence interval (applicable only for type='bootstrap_ci')

    Returns:
    - means: 1D numpy array containing means for each column
    - interval: 1D numpy array containing SEMs or bootstrapped CIs for each column
    """
    nan_mask = ~np.isnan(data)
    
    nanmean_result = np.nanmean(data, axis=0)
    n_valid_values = np.sum(nan_mask, axis=0)
    
    if type == 'sem':
        nanstd_result = np.nanstd(data, axis=0)
        interval = nanstd_result / np.sqrt(n_valid_values)
        
    elif type == 'percentile':
        interval = np.mean(np.array([np.abs(nanmean_result - np.nanpercentile (data, 5, axis=0)), np.abs(nanmean_result - np.nanpercentile (data, 95, axis=0))]))
        
        
    elif type == 'bootstrap':
        n_rows, n_cols = data.shape

        # Initialize array to store bootstrap means
        bootstrap_means = np.zeros((num_samples, n_cols))

        # Perform bootstrap resampling for each column
        for col in range(n_cols):
            bootstrap_samples = np.random.choice(data[:, col][nan_mask[:, col]], size=(num_samples, n_rows), replace=True)
            bootstrap_means[:, col] = np.mean(bootstrap_samples, axis=1)

        # Calculate confidence interval bounds
        ci_lower = np.percentile(bootstrap_means, 100 * (alpha / 2), axis=0)
        ci_upper = np.percentile(bootstrap_means, 100 * (1 - alpha / 2), axis=0)
        
        interval = np.mean([abs(bootstrap_means - ci_lower), abs(bootstrap_means - ci_upper)], axis=0)
        
        interval = np.mean(interval, axis=0)

    else:
        raise ValueError("Invalid 'type' argument. Use either 'sem' or 'bootstrap'.")
    
    return nanmean_result, interval


In [ ]:
# get all the files in the directory
datadir = 'C:/Users/thome/Documents/PYTHON/OFC-CdN 3 state self control/files_for_decoder/'
data_files = glob.glob(os.path.join(datadir, '*.h5'))
file_names = [os.path.basename(file) for file in data_files]

save_dir = 'C:/Users/thome/Documents/PYTHON/OFC-CdN 3 state self control/single_neuron_summary/' 

In [ ]:
# loop over each file
for f_ix, file_path in enumerate(data_files):
    
    print(file_names[f_ix][0:-3]) 

    # access the data for this session
    firing_rates = np.concatenate([pull_from_h5(file_path, 'CdN_zFR'), 
                                pull_from_h5(file_path, 'OFC_zFR')], axis=2)

    u_names = np.concatenate([pull_from_h5(file_path, 'CdN_u_names'), 
                            pull_from_h5(file_path, 'OFC_u_names')], axis=0)

    n_OFC = pull_from_h5(file_path, 'OFC_zFR').shape[2]
    n_CdN = pull_from_h5(file_path, 'CdN_zFR').shape[2]
    brain_areas = np.concatenate([np.zeros(shape=n_CdN, ), np.ones(shape=n_OFC, )]).astype(int)

    ts = pull_from_h5(file_path, 'ts')
    bhv = pd.read_hdf(file_path, key='bhv')

    if len(bhv) > len(firing_rates):
        bhv = bhv.loc[0 :len(firing_rates)-1]

    # subselect trials with a response that was correct
    trials2keep = (bhv['n_sacc'] > 0)
    bhv = bhv.loc[trials2keep]
    firing_rates = firing_rates[trials2keep, :,:]
    firing_rates = np.nan_to_num(firing_rates, nan=0)

    n_trials, n_times, n_units = np.shape(firing_rates)
    
    # get firing rates during choice epoch
    choice_on = np.argwhere(ts == 0)[0][0]
    choice_off = np.argwhere(ts ==300)[0][0]

    cue_on = np.argwhere(ts == -500)[0][0]
    cue_off = np.argwhere(ts == -100)[0][0]

    choice_frs = np.mean(firing_rates[:,choice_on:choice_off,:], axis=1)
    cue_frs = np.mean(firing_rates[:,cue_on:cue_off,:], axis=1)

    ix = (bhv['n_sacc'] ==1)
    n_units = np.size(choice_frs, 1)

    # create indices for the states of each trial
    s1_ix = bhv['state'] == 1
    s2_ix = bhv['state'] == 2
    s3_ix = bhv['state'] == 3

    n_times = len(ts)

    # create factor array for regression analysis where the factors are encoded in a one-hot manner
    one_hot_reg_factors = pd.DataFrame(index=bhv.index)
    one_hot_reg_factors['state_a'] = np.zeros_like(bhv['state'])
    one_hot_reg_factors.loc[bhv['state'] == 1, 'state_a'] = 1
    one_hot_reg_factors['state_b'] = np.zeros_like(bhv['state'])
    one_hot_reg_factors.loc[bhv['state'] == 2, 'state_b'] = 1
    one_hot_reg_factors['state_c'] = np.zeros_like(bhv['state'])
    one_hot_reg_factors.loc[bhv['state'] == 3, 'state_c'] = 1

    one_hot_reg_factors['value'] = bhv['ch_val'].copy()

    one_hot_reg_factors['state_a_val'] = one_hot_reg_factors['state_a'].values * bhv['ch_val'].values
    one_hot_reg_factors['state_b_val'] = one_hot_reg_factors['state_b'].values * bhv['ch_val'].values
    one_hot_reg_factors['state_c_val'] = one_hot_reg_factors['state_c'].values * bhv['ch_val'].values
    one_hot_reg_factors['ch_dir'] = (bhv['side'].values).astype(int)

    # Define the factors (excluding intercept)
    ix = (bhv['n_sacc'] == 1)
    factors = ['state_a', 'state_b', 'state_c', 'value', 'state_a_val', 'state_b_val', 'state_c_val', 'ch_dir']

    n_factors = len(factors)

    # Initialize arrays to store results of time-resolved regression
    t_factor_pvals = np.full((n_units, n_times, n_factors), np.nan)
    t_factor_betas = np.full((n_units, n_times, n_factors), np.nan)


    # initialize an array to accumulate p values from the mean periods into
    choice_pvals = np.zeros((n_units, 3))
    cue_pvals = np.zeros((n_units, 3))

    reg_betas = np.zeros((n_units, 3))
    reg_pvals = np.zeros((n_units, 3))


    # initialize array for accumulating condition mean firing rates into
    f_cond_means = np.zeros((12, n_units))

    # initialize a dataframe for running an anova
    anova_df = pd.DataFrame()
    anova_df['state'] = bhv['state'].loc[ix]
    anova_df['val'] = bhv['ch_val'].loc[ix]

    # loop over the neurons
    for u in tqdm(range(n_units)):
        
        # add the firing rates to the anova
        anova_df['choice_fr'] = choice_frs[ix, u]
        anova_df['cue_fr'] = cue_frs[ix, u]
        
        # run the choice anova
        choice_anova_mdl = pg.anova(dv='choice_fr', between=['state', 'val'], data=anova_df)
        choice_pvals[u,:] = choice_anova_mdl['p-unc'].values[0:3]
        
        # run the cue anova
        cue_anova_mdl = pg.anova(dv='cue_fr', between=['state', 'val'], data=anova_df)
        cue_pvals[u,:] = cue_anova_mdl['p-unc'].values[0:3]
            
        # run value-in-state regressions
        state1_reg = pg.linear_regression(anova_df['val'].loc[ix & s1_ix], anova_df['choice_fr'].loc[ix & s1_ix])
        reg_pvals[u, 0] = state1_reg['pval'].values[1]
        reg_betas[u, 0] = state1_reg['coef'].values[1]
        
        state2_reg = pg.linear_regression(anova_df['val'].loc[ix & s2_ix], anova_df['choice_fr'].loc[ix & s2_ix])
        reg_pvals[u, 1] = state2_reg['pval'].values[1]
        reg_betas[u, 1] = state2_reg['coef'].values[1]
        
        state3_reg = pg.linear_regression(anova_df['val'].loc[ix & s3_ix], anova_df['choice_fr'].loc[ix & s3_ix])
        reg_pvals[u, 2] = state3_reg['pval'].values[1]
        reg_betas[u, 2] = state3_reg['coef'].values[1]

        for t in range(n_times):
            
            # Grab this neuron's firing rate at this time
            one_hot_reg_factors['firing_rate'] = firing_rates[:, t, u]
            
            # Run the regression
            model = smf.ols('firing_rate ~ state_a + state_b + state_c + value + state_a_val + state_b_val + state_c_val + ch_dir', 
                        data=one_hot_reg_factors.loc[ix]).fit()
            
            # Extract p-values and betas for each factor
            for i, factor in enumerate(factors):
                if factor in model.params.index:
                    t_factor_betas[u, t, i] = model.params[factor]
                    t_factor_pvals[u, t, i] = model.pvalues[factor]
        
            
    # # let's try to understand the state code better with an AUROC analysis

    # # initialize arrays to accumulate results into
    # auc_scores = np.zeros((n_units, n_times, 4))
    # shuffle_auc_scores = np.zeros((n_units, n_times, 4)) 

    # # let's only look at the single-saccade trials for this
    # b_triasl2balance = bhv['n_sacc']==1

    # # pull a balanced set of trials
    # trials2use, leftover_ix = pull_balanced_train_set(b_triasl2balance, [bhv['state'].values, bhv['state_cue'].values])

    # # grab the firing rates and behavioral data associated with these trials
    # b_fr = firing_rates[trials2use, :, :]
    # b_state_label = bhv['state'].loc[trials2use]

    # # now loop over each neuron
    # for u in tqdm(range(n_units)):
        
    #     # loop over times
    #     for t in range(n_times):
            
    #         # run one-vs-all AUC classifiers for each state
    #         auc_scores[u, t, 0] = roc_auc_score(b_state_label == 1, b_fr[:,t,u])
    #         auc_scores[u, t, 1] = roc_auc_score(b_state_label == 2, b_fr[:,t,u])
    #         auc_scores[u, t, 2] = roc_auc_score(b_state_label == 3, b_fr[:,t,u])
            
    #         # also look just at the state 1 and 2 trials
    #         auc_scores[u, t, 3] = roc_auc_score(b_state_label[b_state_label < 3] == 1, b_fr[b_state_label < 3,t,u])
            
    #         # run the shuffles
    #         # shuffle the labels
    #         shuff_b_labels = np.random.permutation(b_state_label)
            
    #         # run one-vs-all AUC classifiers for each state
    #         shuffle_auc_scores[u, t, 0] = roc_auc_score(shuff_b_labels == 1, b_fr[:,t,u])
    #         shuffle_auc_scores[u, t, 1] = roc_auc_score(shuff_b_labels == 2, b_fr[:,t,u])
    #         shuffle_auc_scores[u, t, 2] = roc_auc_score(shuff_b_labels == 3, b_fr[:,t,u])
            
    #         # also look just at the state 1 and 2 trials
    #         shuffle_auc_scores[u, t, 3] = roc_auc_score(shuff_b_labels[shuff_b_labels < 3] == 1, b_fr[shuff_b_labels < 3,t,u])            
                              
    # # rectify the scores
    # auc_scores[auc_scores < .5] = 1 - auc_scores[auc_scores < .5]
    # shuffle_auc_scores[shuffle_auc_scores < .5] = 1 - shuffle_auc_scores[shuffle_auc_scores < .5]   
            
    # now save the file
    print('Saving data...')
    save_name = save_dir + file_names[f_ix][0:-3] + '_summary.h5'

    # Open an HDF5 file in write mode ('w' or 'w-' to create or truncate the file)
    with h5py.File(save_name, 'w') as file:
        # Create datasets within the HDF5 file and write data
        file.create_dataset('brain_area', data=brain_areas)  
        file.create_dataset('ts', data=ts)  
        file.create_dataset('t_factor_pvals', data=t_factor_pvals) 
        file.create_dataset('t_factor_betas', data=t_factor_betas)   
        file.create_dataset('valstate_pvals', data=reg_pvals)  
        file.create_dataset('valstate_betas', data=reg_betas)  
        # file.create_dataset('state_auc_scores', data=auc_scores)
        # file.create_dataset('shuffle_auc_scores', data=shuffle_auc_scores)
        
    print('Data saved \n')

print('All files processed :]')

    
            
    